In [11]:
import torch
import argparse
import datasets
from datasets import load_dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import AutoModelForCausalLM, TrainingArguments, HfArgumentParser, AutoTokenizer, TrainerCallback
from huggingface_hub import login
import matplotlib.pyplot as plt
from peft import LoraConfig
import nltk
import numpy as np
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm
import pandas as pd

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
hf_token = os.getenv("HF_TOKEN") # make a .env for this and put your access token as HF_TOKEN=whateverYourAccessTokenIs

In [3]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"
#print("GPU available ", torch.cuda.is_available())

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    token=hf_token
).to(device)

tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)

In [31]:
messages = [
  {"role": "system", "content": "You are a helpful assistant."},
  {"role": "user", "content": "When was UCLA founded?"}
]

def run_model(model, tokenizer, messages, max_new_tokens=5, verbose=False):
    input_text = tokenizer.apply_chat_template(messages, tokenize=False)

    if verbose: print("\n###input_text:###\n", input_text)

    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

    if verbose: print("\n###input_ids:###\n", input_ids)

    terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    output = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        eos_token_id=terminators,
        do_sample=False,
    )


    # Decode the output and return the response without special tokens
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    if verbose: print("\n###response:###\n", response)
    #assistant_response = response.split("\n")[-1].strip()
    assistant_response = response
    return assistant_response

assistant_response = run_model(model=model, tokenizer=tokenizer, messages=messages, max_new_tokens=10, verbose=False)
print(f"\n###Assistant response:###\n{assistant_response}")

c:\Users\Matthew\Desktop\CS 263\LLMvsLeetCode\venv\Lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\Matthew\Desktop\CS 263\LLMvsLeetCode\venv\Lib\site-packages\transformers\generation\configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



###Assistant response:###
system

Cutting Knowledge Date: December 2023
Today Date: 24 Nov 2024

You are a helpful assistant.user

When was UCLA founded?assistant

The University of California, Los


In [14]:
data = pd.read_json("lc_hard.json", lines=False)
data

,desc,skeleton,examples,ref,test
0,\nGiven n non-negative integers representing a...,"\ndef trap(self, height: List[int]) -> int:\n",[],"[\ndef trap(self, height: List[int]) -> int:\n...","{'input': [], 'output': []}"
1,\nGiven an array of integers heights represent...,"\ndef largestRectangleArea(self, heights: List...",[],"[\ndef largestRectangleArea(self, heights: Lis...","{'input': [], 'output': []}"
2,\nGiven two sorted arrays nums1 and nums2 of s...,"\ndef findMedianSortedArrays(self, nums1: List...",[],"[\ndef findMedianSortedArrays(self, nums1: Lis...","{'input': [], 'output': []}"
3,\nGiven two strings s and t of lengths m and n...,"\ndef minWindow(self, s: str, t: str) -> str:\n",[],"[\ndef minWindow(self, s: str, t: str) -> str:...","{'input': [], 'output': []}"
4,"\nYou are given an array of integers nums, the...","\ndef maxSlidingWindow(self, nums: List[int], ...",[],"[\ndef maxSlidingWindow(self, nums: List[int],...","{'input': [], 'output': []}"
5,\nYou are given an array of k linked-lists lis...,\n# Definition for singly-linked list.\n# clas...,[],"[\ndef mergeKLists(self, lists: List[Optional[...","{'input': [], 'output': []}"
6,"\nGiven the head of a linked list, reverse the...",\n# Definition for singly-linked list.\n# clas...,[],"[\ndef reverseKGroup(self, head: Optional[List...","{'input': [], 'output': []}"
7,\nA path in a binary tree is a sequence of nod...,\n# Definition for a binary tree node.\n# clas...,[],"[\ndef traverse(self, node, storer):\n ...","{'input': [], 'output': []}"
8,"\nGiven an integer array nums, return the numb...","\ndef reversePairs(self, nums: List[int]) -> i...",[],"[\ndef reversePairs(self, nums: List[int]) -> ...","{'input': [], 'output': []}"
9,\nGiven an m x n board of characters and a lis...,"\ndef findWords(self, board: List[List[str]], ...",[],"[\ndef findWords(self, board: List[List[str]],...","{'input': [], 'output': []}"


In [27]:
def apply_lc_prompt(desc, skel):
    prompt = (
        "Your task is to complete the following problem in Python. You are provided with a function signature and a description. Output only your completed version of the function. Any additional text beyond a completed function will be deemed incorrect."
        f"Description: {desc}"
        "Below is the starting point for your code. \n"
        f"{skel}"
    )

    return prompt.strip()

In [29]:
dataset = data.copy()
dataset["prompt"] = dataset.apply(lambda x: apply_lc_prompt(x["desc"], x["ref"]), axis=1)
#print(dataset.iloc[0].to_dict())

In [20]:
#import nltk.translate.bleu_score


def eval_bleu(model, tokenizer, dataset, max_new_tokens=1000):
    outputs = []

    for row in tqdm(dataset.to_dict(orient="records")):
        messages = [
            {"role": "system", "content": ""},
            {"role": "user", "content": row["prompt"]},
        ]

        output = run_model(model=model, tokenizer=tokenizer, messages=messages, max_new_tokens=max_new_tokens)

        outputs.append(output)
    
    r, h = [], []
    for idx, row in tqdm(enumerate(dataset.to_dict(orient="records"))):
        refs_in_dataset = row["ref"]
        references = []
        for real_code_solution in refs_in_dataset:
            references.append(real_code_solution.split())
        hypothesis = outputs[idx].split()
        
        r.append(references)
        h.append(hypothesis)
    
    bleu_score = nltk.translate.bleu_score.corpus_bleu(r, h, weights=(1,0,0,0))
    return bleu_score, outputs


In [32]:
df = dataset.copy()
bleu_score, outputs = eval_bleu(model, tokenizer, df)
print(f"Bleu: {bleu_score}")
df["output"] = outputs
display(df)


  0%|          | 0/50 [00:00<?, ?it/s]c:\Users\Matthew\Desktop\CS 263\LLMvsLeetCode\venv\Lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\Matthew\Desktop\CS 263\LLMvsLeetCode\venv\Lib\site-packages\transformers\generation\configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  2%|▏         | 1/50 [00:06<05:28,  6.71s/it]The 

Bleu: 0.13348894565313132


,desc,skeleton,examples,ref,test,prompt,output
0,\nGiven n non-negative integers representing a...,"\ndef trap(self, height: List[int]) -> int:\n",[],"[\ndef trap(self, height: List[int]) -> int:\n...","{'input': [], 'output': []}",Your task is to complete the following problem...,system\n\nCutting Knowledge Date: December 202...
1,\nGiven an array of integers heights represent...,"\ndef largestRectangleArea(self, heights: List...",[],"[\ndef largestRectangleArea(self, heights: Lis...","{'input': [], 'output': []}",Your task is to complete the following problem...,system\n\nCutting Knowledge Date: December 202...
2,\nGiven two sorted arrays nums1 and nums2 of s...,"\ndef findMedianSortedArrays(self, nums1: List...",[],"[\ndef findMedianSortedArrays(self, nums1: Lis...","{'input': [], 'output': []}",Your task is to complete the following problem...,system\n\nCutting Knowledge Date: December 202...
3,\nGiven two strings s and t of lengths m and n...,"\ndef minWindow(self, s: str, t: str) -> str:\n",[],"[\ndef minWindow(self, s: str, t: str) -> str:...","{'input': [], 'output': []}",Your task is to complete the following problem...,system\n\nCutting Knowledge Date: December 202...
4,"\nYou are given an array of integers nums, the...","\ndef maxSlidingWindow(self, nums: List[int], ...",[],"[\ndef maxSlidingWindow(self, nums: List[int],...","{'input': [], 'output': []}",Your task is to complete the following problem...,system\n\nCutting Knowledge Date: December 202...
5,\nYou are given an array of k linked-lists lis...,\n# Definition for singly-linked list.\n# clas...,[],"[\ndef mergeKLists(self, lists: List[Optional[...","{'input': [], 'output': []}",Your task is to complete the following problem...,system\n\nCutting Knowledge Date: December 202...
6,"\nGiven the head of a linked list, reverse the...",\n# Definition for singly-linked list.\n# clas...,[],"[\ndef reverseKGroup(self, head: Optional[List...","{'input': [], 'output': []}",Your task is to complete the following problem...,system\n\nCutting Knowledge Date: December 202...
7,\nA path in a binary tree is a sequence of nod...,\n# Definition for a binary tree node.\n# clas...,[],"[\ndef traverse(self, node, storer):\n ...","{'input': [], 'output': []}",Your task is to complete the following problem...,system\n\nCutting Knowledge Date: December 202...
8,"\nGiven an integer array nums, return the numb...","\ndef reversePairs(self, nums: List[int]) -> i...",[],"[\ndef reversePairs(self, nums: List[int]) -> ...","{'input': [], 'output': []}",Your task is to complete the following problem...,system\n\nCutting Knowledge Date: December 202...
9,\nGiven an m x n board of characters and a lis...,"\ndef findWords(self, board: List[List[str]], ...",[],"[\ndef findWords(self, board: List[List[str]],...","{'input': [], 'output': []}",Your task is to complete the following problem...,system\n\nCutting Knowledge Date: December 202...


In [33]:
print(df.iloc[0].to_dict()["output"])

system

Cutting Knowledge Date: December 2023
Today Date: 24 Nov 2024

user

Your task is to complete the following problem in Python. You are provided with a function signature and a description. Output only your completed version of the function. Any additional text beyond a completed function will be deemed incorrect.Description: \nGiven n non-negative integers representing an elevation map where the width of each bar is 1, compute how much water it can trap after raining.\nBelow is the starting point for your code. 
['\\ndef trap(self, height: List[int]) -> int:\\n        left = 0\\n        right = len(height) - 1\\n        left_max = height[left]\\n        right_max = height[right]\\n        water = 0\\n\\n        while left < right:\\n            if left_max < right_max:\\n                left += 1\\n                left_max = max(left_max, height[left])\\n                water += left_max - height[left]\\n            else:\\n                right -= 1\\n                right_max